# Paraphrase Detection on Twitter

In [1]:
#kamal
import numpy as np
import pandas as pd
import math
import copy
import sys
import sklearn.metrics as skm

#nalla
from itertools import product
from collections import defaultdict
import csv
import nltk
from nltk.corpus import stopwords
import gensim
# from pyemd import emd
from gensim import corpora, models, similarities
from scipy.spatial.distance import euclidean
import pulp

#sowrya
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
from six import iteritems
from collections import deque

## Reading data
* each tweet is considered as a document
* making a list of all the tweets and finding the idf values

In [6]:
def glove_processing():
    f = open('./glove25.txt','r')
    model = {}
    for line in f:
        line = line.split()
        word = line[0]
        embedding = np.array([float(val) for val in line[1:]])
        model[word] =  embedding
    print(len(model))
    return model

In [7]:
df = pd.read_csv('./data/train.data', sep='\t', lineterminator='\n', names=['Topic_Id', 'Topic_Name',  'Sent_1', 'Sent_2', 'Label', 'Sent_1_tag','Sent_2_tag'])
df2 = df[['Sent_1', 'Sent_2', 'Label']]
model = glove_processing()

1193514


## Preprocessing Tweets

* Preprocess the tweets with respect to .... to get a 4D vector

In [8]:
def process_labels(labels):
    '''returns binary labels 0 or 1'''
    new_labels = np.array([])
    for i in labels:
        if type(i) == str and len(i) > 1:
            i = i[1]
        if int(i) >= 3:
            new_labels = np.append(new_labels, int(1))
        else:
            new_labels = np.append(new_labels, int(0))
    return new_labels

In [9]:
train_labels = process_labels(df2.Label)
print(train_labels.shape)

(13063,)


### Cosine Distance for average of sentence vectors

In [10]:
def sent_vec(sent, model):
    ans = np.zeros(25)
    count = 0
    if len(sent) == 0:
        return ans
    for i, word in enumerate(sent.split()):
        if word not in model:
            continue
        else:
            ans += np.array(model[word])
            count += 1
    if count > 0:
        ans = ans / count
    return ans, count


def cosine_dist(vec1, vec2):
    ans = 0
    mod1 = 0
    mod2 = 0
    for i in range(vec1.size):
        ans += vec1[i] * vec2[i]
        mod1 += vec1[i] * vec1[i]
        mod2 += vec2[i] * vec2[i]
    if ans == 0:
        return 0
    return ans / (math.sqrt(mod1) * math.sqrt(mod2))


def cosine_data(df, model):
    arr = np.array([]) #has the cosine distance values for all the tweet pairs
    for i in range(df.Sent_1.size):
        vec1, count1 = sent_vec(df.Sent_1[i], model)
        vec2, count2 = sent_vec(df.Sent_2[i], model)
        ans = cosine_dist(vec1, vec2)
        arr = np.append(arr, ans)
#         if i == 7:
#             print(i, df.Sent_1[i], "sdfsdf", df.Sent_2[i], vec1, vec2, count1, count2, ans)
    return arr

### Word movers distance

In [15]:
singleindexing = lambda m, i, j: m*i+j
unpackindexing = lambda m, k: (k/m, k % m)

def tokens_to_fracdict(tokens):
    cntdict = defaultdict(lambda : 0)
    for token in tokens:
        cntdict[token] += 1
    totalcnt = sum(cntdict.values())
    return {token: float(cnt)/totalcnt for token, cnt in cntdict.items()}

# use PuLP
def word_mover_distance_probspec(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=None):
    all_tokens = list(set(first_sent_tokens+second_sent_tokens))
    wordvecs = {}
    for token in all_tokens:
        try:
            wordvecs[token] = wvmodel[token]
        except KeyError:
            wordvecs[token] = 0

    first_sent_buckets = tokens_to_fracdict(first_sent_tokens)
    second_sent_buckets = tokens_to_fracdict(second_sent_tokens)

    T= pulp.LpVariable.dicts('T_matrix', list(product(all_tokens, all_tokens)), lowBound=0)

    prob = pulp.LpProblem('WMD', sense=pulp.LpMinimize)
    try:
        prob += pulp.lpSum([T[token1, token2]*euclidean(wordvecs[token1], wordvecs[token2])
                        for token1, token2 in product(all_tokens, all_tokens)])
    except KeyError:
        prob+=0
    for token2 in second_sent_buckets:
        prob += pulp.lpSum([T[token1, token2] for token1 in first_sent_buckets])==second_sent_buckets[token2]
    for token1 in first_sent_buckets:
        prob += pulp.lpSum([T[token1, token2] for token2 in second_sent_buckets])==first_sent_buckets[token1]

    if lpFile!=None:
        prob.writeLP(lpFile)
    prob.solve()
    if(prob == None):
        print("yes")
    return prob

def word_mover_distance(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=None):
    prob = word_mover_distance_probspec(first_sent_tokens, second_sent_tokens, wvmodel, lpFile=lpFile)
    return pulp.value(prob.objective)

def arr2_creator(df, model):
    arr2 = np.array([])
    stop_words = stopwords.words('english')
    dist=0
    l = open('./data/train.data','r')
    j = 0
    total = 0
    for i in range(df.Sent_1.size):
        sent1 = df.Sent_1[i].split()
        sent2 = df.Sent_2[i].split()
        sent1 = [w for w in sent1 if w not in stop_words]
        sent2 = [w for w in sent2 if w not in stop_words]
        temp = word_mover_distance(sent1, sent2, model)
        if temp == None:
            temp = total / j
        else:
            total += temp
            j += 1
        arr2 = np.append(arr2, temp)
    return arr2

### Wordnet based noun similarity

In [16]:
def penn_to_wn(tag):
    if tag.startswith('N'):
        return 'n'
    # if tag.startswith('V'):
    #     return 'v'
    # if tag.startswith('J'):
    #     return 'a'
    # if tag.startswith('R'):
    #     return 'r'
    return None

def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
#         print('here')
        return None
    try:
        return wn.synsets(word, wn_tag)[0]
        print('here')
    except:
#         print(wn.synsets(word, wn_tag))
#         print('here_1')
        return None

def shortest_hypernym_paths(synset):
    # hyp = synset.hypernyms()
    # hyp_1 = synset.instance_hypernyms()
    if synset._name == '*ROOT*':
        return {synset: 0}
    queue = deque([(synset,0)])
    path = {}
    while queue:
        s, depth = queue.popleft()
        if s in path:
            continue
        path[s] = depth
        depth += 1
        queue.extend((hyp, depth) for hyp in s.hypernyms())
        queue.extend((hyp, depth) for hyp in s.instance_hypernyms())
    # simulate_root = True
    # # if simulate_root:
    # #     fake_synset = wn.synset(None)
    # #     fake_synset._name = '*ROOT*'
    # #     path[fake_synset] = max(path.values()) + 1
    return path

def shortest_path_distance(synset_1,synset_2):
    if synset_1 == synset_2:
        return 0
    dist_dict1 = shortest_hypernym_paths(synset_1)
    dist_dict2 = shortest_hypernym_paths(synset_2)
    inf = float('inf')
    path_distance = inf
    for synset, d1 in iteritems(dist_dict1):
        d2 = dist_dict2.get(synset, inf)
        path_distance = min(path_distance, d1 + d2)
    return None if math.isinf(path_distance) else path_distance

def path_similarity(synset_1,synset_2):
    distance = shortest_path_distance(synset_1,synset_2)
    if distance is None or distance < 0:
        return None
    return 1.0/(distance + 1)

def sentence_similarity(sentence1, sentence2):
    sentence1, sentence2 = pos_tag(word_tokenize(sentence1)), pos_tag(word_tokenize(sentence2))
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
    synsets2 = [ss for ss in synsets2 if ss]
    score, count = 0.0, 0
    for syn1 in synsets1:
        arr_simi_score = []
        for syn2 in synsets2:
            simi_score = path_similarity(syn1,syn2)
            if simi_score is not None:
                arr_simi_score.append(simi_score)
        if(len(arr_simi_score) > 0):
            best = max(arr_simi_score)
            score += best
            count += 1
    if count == 0:
        score = 0
    else:
        score /= count
    return score

def arr3_creator(df):
    arr3 = np.array([])
    s_1 = "EJ Manuel the 1st QB to go in this draft"
    s_2 = "if EJ is the 1st QB off the board"
    for i in range(df.Sent_1.size):
#     val = (sentence_similarity(s_2, s_1) + sentence_similarity(s_1, s_2))/2
        val = (sentence_similarity(df.Sent_1[i], df.Sent_2[i]) + sentence_similarity(df.Sent_2[i], df.Sent_1[i]))/2
#         print(df.Sent_1[i], df.Sent_2[i], val)
        arr3 = np.append(arr3, val)
    return arr3

### Creating vectors
* Appending all values for each tweet pair to create a 4D vector 

In [17]:
arr1 = cosine_data(df2, model)
print(arr1.shape)

(13063,)


In [18]:
arr2 = arr2_creator(df2, model)
print(arr2.shape)

(13063,)


In [19]:
arr3 = arr3_creator(df2)
print(arr3.shape)

(13063,)


In [20]:
def create_vectors(arr1, arr2, arr3):
    new_arr = np.vstack((arr1, arr2, arr3))
    return new_arr

In [21]:
train_data = create_vectors(arr1, arr2, arr3) #(3,N) dimensions
print(train_data.shape)
#each column is a vector

(3, 13063)


## Training Models

### Logistic Regression

In [14]:
def loss(model, train_data, train_labels):
    ans = 0
    for i in range(train_labels.size):
#         print(np.dot(model, train_data[:, i]))
        val = sigmoid(np.dot(model, train_data[:, i]))
        ans += (train_labels[i] * np.log(val)) + ((1-train_labels[i]) * np.log(1-val))
    return abs(ans)

def classify(val):
    if val >= 0.5:
        return 1
    else:
        return 0

def sigmoid(val):
    ans = 1 / (1 + np.exp(-val))
    return ans

def loss_gradient(model, train_data, train_labels):
    loss = np.zeros(model.size)
    for j in range(train_labels.size):
#         print(j, train_data[:, j])
        loss = loss + train_data[:, j] * (sigmoid(np.dot(model, train_data[:, j])) - train_labels[j])
#         if loss[1] == None:
#             print (j, train_data[:,j])
    return loss

def log_reg(train_data, train_labels):
    '''returns a trained model'''
    eta = 0.00005
    model = np.random.rand(3)
    curr_loss = loss(model, train_data, train_labels)
    prev_loss = curr_loss - 100
    while abs(curr_loss - prev_loss) > 1:
        print(abs(curr_loss - prev_loss))
#     while loss(model, train_data, train_labels) > 0.1:
        val = loss_gradient(model, train_data, train_labels)
        model -= eta * val
        prev_loss = curr_loss
        curr_loss = loss(model, train_data, train_labels)
    return model

In [91]:
lr_model = log_reg(train_data, train_labels)
# print(lr_model)

100.0
356.67451460531447
294.4462294464229
242.65025904731738
199.8089443264289
164.52644437112394
135.5477402972665
111.78143966988682
92.30007592569746
76.3280318927882
63.22383346072638
52.46092235575543
43.60918245888479
36.31831838699418
30.30348345308994
25.333166337644798
21.219152984859647
17.80830179759505
14.975854779760994
12.620023725648025
10.657620813568428
9.020537134195365
7.652905689552426
6.508814949588668
5.550464386530621
4.746674543319386
4.071681512787109
3.50415974133648
3.026428348767695
2.623805175772759
2.284079966342688
1.9970838115696097
1.7543365360761527
1.5487573301370503
1.3744268129212287
1.226391009834515
1.1004995631810743
[-0.62283918  1.17060066 -0.7749008 ]


In [16]:
print(lr_model)

[-0.17261612 -0.31880689  0.39960586]


# testing

In [22]:
df = pd.read_csv('./data/test.data', sep='\t', lineterminator='\n', names=['Topic_Id', 'Topic_Name',  'Sent_1', 'Sent_2', 'Label', 'Sent_1_tag','Sent_2_tag'])
test_pairs = df[['Sent_1', 'Sent_2']]
test_labels = process_labels(df.Label)

In [23]:
test_arr1 = cosine_data(test_pairs, model)
print(test_arr1.shape)

(972,)


In [24]:
test_arr2 = arr2_creator(test_pairs, model)
print(test_arr2.shape)

(972,)


In [25]:
test_arr3 = arr3_creator(test_pairs)
print(test_arr3.shape)

(972,)


In [26]:
test_data = create_vectors(test_arr1, test_arr2, test_arr3)

In [27]:
def testing(model, test_data, test_labels):
    predict = np.array([])
    for i in range(test_labels.size):
        val = classify(sigmoid(np.dot(model, test_data[:, i])))
        predict = np.append(predict, val)
    return predict

In [41]:
predicted_arr = testing(lr_model, test_data, test_labels)
print(skm.classification_report(test_labels, predicted_arr))

In [79]:
print(skm.accuracy_score(test_labels, predicted_arr))
# print(train_data[2])

0.6820987654320988
[-1.75727089 -1.75727089 -1.75727089 ... -1.75727089 -1.75727089
 -1.75727089]


## MLP


In [30]:
from keras.models import Sequential
from keras.utils import to_categorical
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout

Using TensorFlow backend.


In [38]:
def MLP(train_data, train_labels, test_data, test_labels):
    X_train = train_data.T
    y_train = train_labels
    y_binary = to_categorical(y_train)

    scale = np.max(X_train)
    X_train /= scale

    mean = np.std(X_train)
    X_train -= mean

    input_dim = X_train.shape[1]
#     print?(X_train[0])
#     print(input_dim)
    nb_classes = 1
    
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(nb_classes))
    model.add(Activation('sigmoid'))
#     model = Sequential()
#     model.add(Dense(64, input_dim=input_dim, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(1, activation='sigmoid'))

    # we'll use categorical xent for the loss, and RMSprop as the optimizer
    model.compile(loss='mse', optimizer='rmsprop')

    print("Training...")
    model.fit(X_train, y_train, nb_epoch=500, batch_size=128, verbose=2)
    
    X_test = test_data.T
    y_test = test_labels
    preds = model.predict_classes(X_test, verbose=0)

    score = model.evaluate(X_test, y_test, batch_size=128)
    print("score",score)
    return preds, model
# print("Generating test predictions...")
# preds = model.predict_classes(X_test, verbose=0)

In [39]:
preds = np.array([])
preds, mlp_model = MLP(train_data, train_labels, test_data, test_labels)
# print(mlp_model)

Training...
Epoch 1/500


/home/kamal/.local/lib/python3.5/site-packages/ipykernel_launcher.py:37: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


 - 0s - loss: 0.2030
Epoch 2/500
 - 0s - loss: 0.1956
Epoch 3/500
 - 0s - loss: 0.1947
Epoch 4/500
 - 0s - loss: 0.1940
Epoch 5/500
 - 0s - loss: 0.1933
Epoch 6/500
 - 0s - loss: 0.1930
Epoch 7/500
 - 0s - loss: 0.1922
Epoch 8/500
 - 0s - loss: 0.1926
Epoch 9/500
 - 0s - loss: 0.1922
Epoch 10/500
 - 0s - loss: 0.1923
Epoch 11/500
 - 0s - loss: 0.1920
Epoch 12/500
 - 0s - loss: 0.1914
Epoch 13/500
 - 0s - loss: 0.1918
Epoch 14/500
 - 0s - loss: 0.1910
Epoch 15/500
 - 0s - loss: 0.1912
Epoch 16/500
 - 0s - loss: 0.1910
Epoch 17/500
 - 0s - loss: 0.1908
Epoch 18/500
 - 0s - loss: 0.1913
Epoch 19/500
 - 0s - loss: 0.1906
Epoch 20/500
 - 0s - loss: 0.1906
Epoch 21/500
 - 0s - loss: 0.1908
Epoch 22/500
 - 0s - loss: 0.1908
Epoch 23/500
 - 0s - loss: 0.1906
Epoch 24/500
 - 0s - loss: 0.1906
Epoch 25/500
 - 0s - loss: 0.1904
Epoch 26/500
 - 0s - loss: 0.1902
Epoch 27/500
 - 0s - loss: 0.1905
Epoch 28/500
 - 0s - loss: 0.1895
Epoch 29/500
 - 0s - loss: 0.1904
Epoch 30/500
 - 0s - loss: 0.1900
E

Epoch 239/500
 - 0s - loss: 0.1870
Epoch 240/500
 - 0s - loss: 0.1866
Epoch 241/500
 - 0s - loss: 0.1871
Epoch 242/500
 - 0s - loss: 0.1871
Epoch 243/500
 - 0s - loss: 0.1875
Epoch 244/500
 - 0s - loss: 0.1869
Epoch 245/500
 - 0s - loss: 0.1869
Epoch 246/500
 - 0s - loss: 0.1873
Epoch 247/500
 - 0s - loss: 0.1871
Epoch 248/500
 - 0s - loss: 0.1873
Epoch 249/500
 - 0s - loss: 0.1868
Epoch 250/500
 - 0s - loss: 0.1872
Epoch 251/500
 - 0s - loss: 0.1871
Epoch 252/500
 - 0s - loss: 0.1868
Epoch 253/500
 - 0s - loss: 0.1874
Epoch 254/500
 - 0s - loss: 0.1871
Epoch 255/500
 - 0s - loss: 0.1870
Epoch 256/500
 - 0s - loss: 0.1870
Epoch 257/500
 - 0s - loss: 0.1870
Epoch 258/500
 - 0s - loss: 0.1872
Epoch 259/500
 - 0s - loss: 0.1868
Epoch 260/500
 - 0s - loss: 0.1872
Epoch 261/500
 - 0s - loss: 0.1871
Epoch 262/500
 - 0s - loss: 0.1866
Epoch 263/500
 - 0s - loss: 0.1869
Epoch 264/500
 - 0s - loss: 0.1872
Epoch 265/500
 - 0s - loss: 0.1874
Epoch 266/500
 - 0s - loss: 0.1870
Epoch 267/500
 - 0s 

 - 0s - loss: 0.1862
Epoch 474/500
 - 0s - loss: 0.1864
Epoch 475/500
 - 0s - loss: 0.1866
Epoch 476/500
 - 0s - loss: 0.1867
Epoch 477/500
 - 0s - loss: 0.1860
Epoch 478/500
 - 0s - loss: 0.1867
Epoch 479/500
 - 0s - loss: 0.1864
Epoch 480/500
 - 0s - loss: 0.1865
Epoch 481/500
 - 0s - loss: 0.1870
Epoch 482/500
 - 0s - loss: 0.1867
Epoch 483/500
 - 0s - loss: 0.1860
Epoch 484/500
 - 0s - loss: 0.1866
Epoch 485/500
 - 0s - loss: 0.1863
Epoch 486/500
 - 0s - loss: 0.1867
Epoch 487/500
 - 0s - loss: 0.1869
Epoch 488/500
 - 0s - loss: 0.1867
Epoch 489/500
 - 0s - loss: 0.1862
Epoch 490/500
 - 0s - loss: 0.1869
Epoch 491/500
 - 0s - loss: 0.1868
Epoch 492/500
 - 0s - loss: 0.1864
Epoch 493/500
 - 0s - loss: 0.1863
Epoch 494/500
 - 0s - loss: 0.1862
Epoch 495/500
 - 0s - loss: 0.1863
Epoch 496/500
 - 0s - loss: 0.1868
Epoch 497/500
 - 0s - loss: 0.1862
Epoch 498/500
 - 0s - loss: 0.1866
Epoch 499/500
 - 0s - loss: 0.1868
Epoch 500/500
 - 0s - loss: 0.1870
972/972 [=========================

In [40]:
# predicted_arr = testing(mlp_model, test_data, test_labels)
print(skm.classification_report(test_labels, preds))
print(skm.accuracy_score(test_labels, preds))

              precision    recall  f1-score   support

         0.0       0.71      0.97      0.82       663
         1.0       0.69      0.17      0.27       309

   micro avg       0.71      0.71      0.71       972
   macro avg       0.70      0.57      0.54       972
weighted avg       0.71      0.71      0.64       972

0.7109053497942387
